# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
csv_path = "../output_data/less_humid.csv"
cities_df = pd.read_csv(csv_path)
cities_df

,City,Country,Cloudiness,Humidity,Latitude,Longitude,Temperature,Max Temp,Windspeed
0,Guarapuava,BR,100,99,-25.40,-51.46,58.93,58.93,8.93
1,Gari,RU,100,98,59.43,62.35,50.18,50.18,5.50
2,Verkhotur'ye,RU,78,97,58.86,60.81,49.05,49.05,4.56
3,Puteyets,RU,100,97,65.17,57.09,50.79,50.79,3.83
4,Nantai,CN,100,97,40.92,122.80,68.07,68.07,11.07
...,...,...,...,...,...,...,...,...,...
550,Bilma,NE,56,14,18.69,12.92,98.98,98.98,4.63
551,Aripuanã,BR,0,14,-9.17,-60.63,104.23,104.23,3.47
552,Aswān,EG,0,14,24.09,32.91,92.95,92.95,12.15
553,Saint George,US,1,13,37.10,-113.58,105.26,107.01,7.34


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key = g_key)

# Store 'Lat' and 'Lng' into  locations
locations = cities_df[["Latitude", "Longitude"]].astype(float)

# Store 'humidity' as weight
humid_data = cities_df['Humidity'].astype(float)

In [4]:
# Create heatmap
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humid_data, dissipating=False, max_intensity=100, point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
narrowed_df = cities_df.loc[(cities_df["Windspeed"] <= 10) & (cities_df["Cloudiness"] == 0) & \
                                   (cities_df["Max Temp"] >= 70) & (cities_df["Max Temp"] <= 80)].dropna()
narrowed_df

,City,Country,Cloudiness,Humidity,Latitude,Longitude,Temperature,Max Temp,Windspeed
8,Lata,PT,0,96,40.16,-8.33,71.89,73.99,4.00
63,Vrangel',RU,0,92,42.73,133.08,70.21,70.21,4.45
65,Peniche,PT,0,91,39.36,-9.38,69.67,70.00,5.99
97,Kodarmā,IN,0,88,24.47,85.60,79.34,79.34,4.50
299,Camacha,PT,0,73,33.08,-16.33,75.20,75.20,9.17
310,Ponta do Sol,PT,0,72,32.67,-17.10,69.39,72.00,2.08
340,Kigoma,TZ,0,69,-4.88,29.63,73.40,73.40,5.53
351,Alanya,TR,0,69,36.54,32.00,77.00,77.00,2.24
352,Poso,ID,0,68,-1.40,120.75,75.02,75.02,5.84
425,Kindu,CD,0,59,-2.95,25.95,78.60,78.60,2.13


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# Create a hotel_df
hotel_df = narrowed_df.loc[:,["City", "Country", "Latitude", "Longitude"]].rename(
    columns = {'Latitude':'Lat', 'Longitude':'Lng'})

# Add a "Hotel Name" column to the DataFrame.
hotel_df['Hotel Name'] = ''

# Display the result
hotel_df

,City,Country,Lat,Lng,Hotel Name
8,Lata,PT,40.16,-8.33,
63,Vrangel',RU,42.73,133.08,
65,Peniche,PT,39.36,-9.38,
97,Kodarmā,IN,24.47,85.60,
299,Camacha,PT,33.08,-16.33,
310,Ponta do Sol,PT,32.67,-17.10,
340,Kigoma,TZ,-4.88,29.63,
351,Alanya,TR,36.54,32.00,
352,Poso,ID,-1.40,120.75,
425,Kindu,CD,-2.95,25.95,


In [8]:
# base url
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?" 
params = {
    'types': 'lodging',
    'keyword': 'hotel',
    'radius':5000,
    'key': g_key
}

for index, row in hotel_df.iterrows():
    lat = row['Lat']
    lng = row['Lng']
    params['location']= f"{lat},{lng}"
    response = requests.get(url, params=params).json()
    try:
        hotel_df.append(response['response'][0]['name'])
    except:
        pass

In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))